## Beginning Steps

In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
from datetime import datetime
import argparse
import psutil
import numpy as np
import logging
import numpy as np
import tensorflow as tf
import time
import glob
from tensorflow.python.framework import ops
import math
from dotenv import find_dotenv, load_dotenv
import ftplib


nb_dir = os.path.join(Path(os.getcwd()).parents[0], 'src', 'data')
if nb_dir not in sys.path:
    sys.path.insert(0, nb_dir)
print(sys.path)
import features_selection as fs
import make_dataset as md
import build_data as bd
import get_raw_data as grd
#import data_classes

models_dir = os.path.join(Path(os.getcwd()).parents[0], 'src', 'models')
if models_dir not in sys.path:
    sys.path.insert(0, models_dir)
import nn_real as nn

try:
    import horovod.tensorflow as hvd
except:
    print("Failed to import horovod module. "
          "%s is intended for use with Uber's Horovod distributed training "
          "framework. To create a Docker image with Horovod support see "
          "docker-examples/Dockerfile.horovod." % __file__)
    raise

load_dotenv(find_dotenv())

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
2018-11-20 18:20:27,516 - matplotlib - DEBUG - CACHEDIR=/home/ubuntu/.cache/matplotlib
2018-11-20 18:20:27,519 - matplotlib.font_manager - DEBUG - Using fontManager instance from /home/ubuntu/.cache/matplotlib/fontList.json


['/home/ubuntu/MLMortgage/src/data', '', '/home/ubuntu/src/cntk/bindings/python', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python36.zip', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/lib-dynload', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/extensions', '/home/ubuntu/.ipython']


2018-11-20 18:20:27,658 - matplotlib.backends - DEBUG - backend module://ipykernel.pylab.backend_inline version unknown


True

2018-11-20 18:20:27,900 - matplotlib.backends - DEBUG - backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
RAW_DIR = os.path.join(Path(os.getcwd()).parents[0], 'data', 'raw') 
PRO_DIR = os.path.join(Path(os.getcwd()).parents[0], 'data', 'processed')
RANDOM_SEED = 123  # Set the seed to get reproducable results.
DT_FLOAT = tf.float32
NP_FLOAT = np.dtype('float32')

print(RAW_DIR, PRO_DIR)

/home/ubuntu/MLMortgage/data/raw /home/ubuntu/MLMortgage/data/processed


In [3]:
hvd.init()

0

## Defining FLAGS

In [4]:
def FLAGS_setting(FLAGS, net_number):
    # To determine an optimal set of hyperparameters, see Section 11.4.2 of the
    # deep learning book. Has (1) grid, (2) random, and (3) Bayesian
    # model-based search methods.Swersky et al. have a paper mentioned in that
    # section (published in 2014).

    # Hyperparameters
    # FLAGS.epoch_num = 2  # 14  # 17  # 35  # 15
    #print("FLAGS.epoch_num", FLAGS.epoch_num)
    # FLAGS.batch_size = 141600 # 4425 # 4000  
    FLAGS.dropout_keep = 0.9  # 0.9  # 0.95  # .75  # .6
    # ### parameters for training optimizer.
    #FLAGS.learning_rate = .1  # .075  # .15  # .25
    FLAGS.momentum = .5  # used by the momentum SGD.

    # ### parameters for inverse_time_decay
    FLAGS.decay_rate = 1
    FLAGS.decay_step = 800 * 4400 #steps_per_epoch 1 * 80000 #according to paper: 800 epochs
    FLAGS.rate_min = .0015
    # ### parameters for exponential_decay
    # FLAGS.decay_base = .96  # .96
    # FLAGS.decay_step = 15000  # 12320  # 4 * 8700

    # ### parameters for regularization
    FLAGS.reg_rate = .01 * 1e-3  # * 1e-3

    FLAGS.batch_norm = True  # False  #
    FLAGS.dropout = True
    # A flag to show the results on the held-out test set. Keep this at False.
    FLAGS.test_flag = True
    FLAGS.xla = True  # False
    FLAGS.stratified_flag = False
    #FLAGS.batch_type = 'batch'    
    FLAGS.weighted_sampling = False  # True  #
    # FLAGS.logdir =  os.path.join(Path.home(), 'real_summaries')  # 
    #FLAGS.n_hidden = 3
    #FLAGS.s_hidden = [200, 140, 140]
    # FLAGS.allow_summaries = False
    FLAGS.epoch_flag = 0    
    
    #FLAGS.max_epoch_size = 141600*70 #137 # -1
    
    FLAGS.valid_batch_size = 1200000
    FLAGS.test_batch_size = 1200000
    
    FLAGS.train_dir = 'chuncks_random_c1millx2_train'
    FLAGS.valid_dir = 'chuncks_random_c1millx2_valid'
    FLAGS.test_dir = 'chuncks_random_c1millx2_test'
    FLAGS.train_period=[121,323] #[121,279] #[121, 143] 
    FLAGS.valid_period=[324,329] #[280,285] #[144, 147] 
    FLAGS.test_period=[330,342] #[286,304] #[148, 155]

    FLAGS.epoch_num=15 
    FLAGS.max_epoch_size=-1 
    FLAGS.batch_size=4425*2 # two files per worker except at master!
    FLAGS.lr_decay_policy       = 'time'
    FLAGS.lr_decay_epochs       = 30
    FLAGS.lr_decay_rate         = 0.1
    FLAGS.lr_poly_power         = 2.
    FLAGS.eval = False # True=Evaluation else Training
    FLAGS.save_interval = 450
    FLAGS.nstep_burnin = 20 # step from to count consuming time for a batch
    FLAGS.summary_interval = 1800 # Time in seconds between saves of summary statistics
    FLAGS.display_every = 100 # How often (in iterations) to print out running information
    FLAGS.total_examples = 52000000 #-1 to training all dataset, otherwise the training will have a fixed length
    
    #Retrieveng from ftp:
    FLAGS.ftp_dir = 'processed/c1mill'
    
    
    if FLAGS.n_hidden < 0 : raise ValueError('The size of hidden layer must be at least 0')
    if (FLAGS.n_hidden > 0) and (FLAGS.n_hidden != len(FLAGS.s_hidden)) : raise ValueError('Sizes in hidden layers should match!')
    
    if (net_number==0):
        FLAGS.name ='default_settings'        
    elif (net_number==1):
        FLAGS.name ='Xworkers_1mill'
        FLAGS.batch_layer_type = 'batch'        
        
    return FLAGS

In [5]:
import tensorflow as tf

FLAGS, UNPARSED = nn.update_parser(argparse.ArgumentParser())
print("UNPARSED", UNPARSED)
FLAGS.logdir = Path(str('/home/ubuntu/summ_15ep_2wrk/'))
if not os.path.exists(os.path.join(FLAGS.logdir)): #os.path.exists
    os.makedirs(os.path.join(FLAGS.logdir))
else:
    print('existent directory')
FLAGS = FLAGS_setting(FLAGS, 1)

UNPARSED ['-f', '/run/user/1000/jupyter/kernel-c1931a77-68e2-494b-8a20-4401b661d05a.json']


In [6]:
print("FLAGS", FLAGS) #you can change the FLAGS by adding the setting before this line.

FLAGS Namespace(allow_summaries=True, batch_layer_type='batch', batch_norm=True, batch_size=8850, batch_type='layer', data_dir='/input_data', decay_rate=1, decay_step=3520000, display_every=100, dropout=True, dropout_keep=0.9, epoch_flag=0, epoch_num=15, eval=False, ftp_dir='processed/c1mill', learning_rate=0.1, log_file=None, logdir=PosixPath('/home/ubuntu/summ_15ep_2wrk'), loss_tolerance=0.0001, lr_decay_epochs=30, lr_decay_policy='time', lr_decay_rate=0.1, lr_poly_power=2.0, max_epoch_size=-1, momentum=0.5, n_hidden=3, name='Xworkers_1mill', nstep_burnin=20, rate_min=0.0015, reg_rate=1e-05, s_hidden=[200, 140, 140], save_interval=450, stratified_flag=False, summary_interval=1800, test_batch_size=1200000, test_dir='chuncks_random_c1millx2_test', test_flag=True, test_period=[330, 342], total_examples=52000000, train_dir='chuncks_random_c1millx2_train', train_period=[121, 323], valid_batch_size=1200000, valid_dir='chuncks_random_c1millx2_valid', valid_period=[324, 329], weighted_sampli

## Network Builder, Trainer

In [7]:
class GPUNetworkBuilder(object):
    """This class provides convenient methods for constructing feed-forward
    networks with internal data layout of 'NCHW'.
    """
    def __init__(self,
                 # is_training,
                 dtype=DT_FLOAT,
                 activation='RELU',
                 use_batch_norm=True,
                 batch_norm_config = {'decay':   0.9,
                                      'epsilon': 1e-4,
                                      'scale':   True,
                                      'zero_debias_moving_mean': False}):
        self.dtype             = dtype
        self.activation_func   = activation
        # self.is_training       = is_training
        self.use_batch_norm    = use_batch_norm
        self.batch_norm_config = batch_norm_config
        #self._layer_counts     = defaultdict(lambda: 0)        
        
    def variable_summaries(self, name, var, allow_summaries):
        """Create summaries for the given Tensor (for TensorBoard visualization (TB graphs)).
            Calculate the mean, min, max, histogram and standardeviation for 'var' variable and save the information
            in tf.summary.

        Args: 
             name (String): the of the scope for summaring. For min, max and standardeviation 'calculate_std' is used as sub-scope.
             var (Tensor): This is the tensor variable for building summaries.
        Returns: 
            None
        Raises:        
        """
        if allow_summaries:
            with tf.name_scope(name):
                mean = tf.reduce_mean(var)
                tf.summary.scalar('mean', mean)
                with tf.name_scope('calculate_std'):
                    stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
                tf.summary.scalar('stddev', stddev)
                tf.summary.scalar('min', tf.reduce_min(var))
                tf.summary.scalar('max', tf.reduce_max(var))
                tf.summary.histogram('histogram', var)
                
    def _variable_on_cpu(self, name,
                     shape,
                     initializer=None,
                     regularizer=None,
                     dtype=DT_FLOAT):
        """Create a Variable or get an existing one stored on CPU memory.    

        Args:
            name (String): name of the variable.
            shape (list of ints): Shape of the variable.
            initializer: Default None. Initializer for Variable.
            regularizer (A (Tensor -> Tensor or None) function): Default None. Regularizer for Variable.
            dtype (TYPE): Type of the new variable.
        Returns:
            Variable Tensor
        """
        with tf.device('/gpu:1'): # this operation is assigned to this device, but this make a copy of data when is transferred on and off the device, which is expensive.
            var = tf.get_variable(
                name,
                shape,
                initializer=initializer,
                regularizer=regularizer,
                dtype=dtype)
        return var

    def _create_variable(self, name,
                         shape, allow_summaries, 
                         initializer=None,
                         regularizer=None,
                         dtype=DT_FLOAT):
        """Call _variable_on_cpu methods and variable_summaries for the 'name' tensor variable. 

        Args:
            name (String): name of the variable.
            shape (list of ints): Shape of the variable.
            initializer: Default None. Initializer for Variable.
            regularizer (A (Tensor -> Tensor or None) function): Default None. Regularizer for Variable.
            dtype (TYPE): Type of the new variable.
        Returns:
            Variable Tensor
        """
        var = self._variable_on_cpu(name, shape, initializer, regularizer, dtype)
        self.variable_summaries(name + '/summaries', var, allow_summaries)
        return var

    def create_weights(self, name, shape, reg_rate, allow_summaries):
        """Create a Variable initialized with weights which are truncated normal distribution and regularized by
        l1_regularizer (L1 regularization encourages sparsity, Regularization can help prevent overfitting).    

        Args:
            name (String): name of the variable.
            shape (list of ints): Shape of the variable.
        Returns:
            Variable Tensor
        """    
        dtype = DT_FLOAT
        # kernel_initializer = tf.uniform_unit_scaling_initializer(
        #     factor=1.43, dtype=DT_FLOAT)
        # kernel_initializer = tf.contrib.layers.xavier_initializer(
        #     uniform=True, dtype=DT_FLOAT)
        kernel_initializer = tf.truncated_normal_initializer(
            stddev=(1.0 / np.sqrt(shape[0])), dtype=dtype)

        regularizer = tf.contrib.layers.l2_regularizer(
            np.float32(reg_rate), 'penalty')
        return self._create_variable(name, shape, allow_summaries, kernel_initializer, regularizer,
                                dtype)

    def bias_variable(self, name, shape, layer_name, weighted_sampling): # FLAGS.weighted_sampling
        """Create a bias variable with appropriate initialization. In case of FLAGS.weighted_sampling==False
        and layer_name contains 'soft' the bias variable will contain a np.array of Negative values. Otherwise
        the bias variable will be initialized in zero.

        Args:
            name (String): name of the variable.
            shape (list of ints): Shape of the variable.
            layer_name (String): name of the layer.
        Returns:
            Variable Tensor.
        """
        def initial_bias(layer_name):
            """Get the initial value for the bias of the layer with layer_name."""
            if (not weighted_sampling) and 'soft' in layer_name:
                return np.array(
                    [-4.66, -3.81, -4.81, -3.90, -0.08, -3.90, -7.51],
                    dtype=NP_FLOAT) + NP_FLOAT(4.1)
            return 0.0

        initial_value = initial_bias(layer_name)
        with tf.name_scope(name) as scope:
            initial = tf.constant(initial_value, shape=shape)
            bias = tf.Variable(initial, name=scope)
            self.variable_summaries('summaries', bias)
        return bias        
    
    def dropout_layer(self, name, tensor_before, FLAGS):
        """Compute dropout to tensor_before with name scoping and a placeholder for keep_prob. 
        With probability keep_prob, outputs the input element scaled up by 1 / keep_prob, otherwise outputs 0.

        Args:
            name (String): name of the scope.
            tensor_before (Tensor): Variable Tensor.        
        Returns:
            Variable Tensor of the same shape of tensor_before.
        """   
        if not FLAGS.dropout:
            print('There is not dropout for' + name)
            return tensor_before
        with tf.name_scope(name) as scope:
            keep_prob = tf.placeholder(DT_FLOAT, None, name='keep_proba')
            tf.summary.scalar('keep_probability', keep_prob)
            dropped = tf.nn.dropout(tensor_before, keep_prob=keep_prob, name=scope)
            self.variable_summaries('input_dropped_out', dropped, FLAGS.allow_summaries)
        return dropped

    def batch_normalization(self, name, input_tensor, train_flag, FLAGS):
        """Perform batch normalization over the input tensor.
        Batch normalization helps avoid overfitting and we're able to use more
        aggressive (larger) learning rates, resulting in faster convergence.
        training parameter: Either a Python boolean, or a TensorFlow boolean scalar tensor (e.g. a placeholder). 
        Whether to return the output in training mode (normalized with statistics of the current batch) or in 
        inference mode (normalized with moving statistics). NOTE: make sure to set this parameter correctly, 
        or else your training/inference will not work properly.

        Args:
            name (String): name of the scope and the name of the layer.
            input_tensor (Tensor): Variable Tensor.        
        Returns:
            Variable Tensor # the same shape of input_tensor??.
        """
        # if not FLAGS.batch_norm:
        #     return input_tensor
        # train_flag = tf.get_default_graph().get_tensor_by_name('train_flag:0')
        with tf.name_scope(name):
            normalized = tf.layers.batch_normalization(
                input_tensor,
                center=True,
                scale=True,
                training=train_flag,
                name=name)  # renorm=True, renorm_momentum=0.99)
            self.variable_summaries('normalized_batch', normalized, FLAGS.allow_summaries)
        return normalized

    def layer_normalization(self, name, input_tensor, FLAGS):
        """Perform layer normalization.

        Layer normalization helps avoid overfitting and we're able to use more
        aggressive (larger) learning rates, resulting in faster convergence.
        Can be used as a normalizer function for conv2d and fully_connected.

        Given a tensor inputs of rank R, moments are calculated and normalization 
        is performed over axes begin_norm_axis ... R - 1. 
        Scaling and centering, if requested, is performed over axes begin_params_axis .. R - 1.
        """
        # if not FLAGS.batch_norm:
        #     return input_tensor
        with tf.name_scope(name):
            normalized = tf.contrib.layers.layer_norm(
                input_tensor, center=True, scale=True, scope=name)
            self.variable_summaries('normalized_layer', normalized, FLAGS.allow_summaries)
        return normalized


    def normalize(self, name, input_tensor, train_flag, FLAGS):
        """Perform either type (batch/layer) of normalization."""
        if not FLAGS.batch_norm:
            return input_tensor
        if FLAGS.batch_type.lower() == 'batch':
            return self.batch_normalization(name, input_tensor, train_flag, FLAGS)
        if FLAGS.batch_type.lower() == 'layer':
            return self.layer_normalization(name, input_tensor, FLAGS)
        raise ValueError('Invalid value for batch_type: ' + FLAGS.batch_type)

    def nn_layer(self, input_tensor, output_dim, layer_name, FLAGS, act, train_flag):
        """Create a simple neural net layer.

        It performs the affine transformation and uses the activation function to
        nonlinearize. It further sets up name scoping so that the resultant graph
        is easy to read, and adds a number of summary ops.
        """
        input_dim = input_tensor.shape[1].value    
        with tf.variable_scope(layer_name): # A context manager for defining ops that creates variables (layers).
            weights = self.create_weights('weights', [input_dim, output_dim], FLAGS.reg_rate, FLAGS.allow_summaries)
            # This is outdated and no longer applies: Do not change the order of
            # batch normalization and drop out. batch # normalization has to stay
            # __before__ the drop out layer.
            self.variable_summaries('input', input_tensor, FLAGS.allow_summaries)
            input_tensor = self.dropout_layer('dropout', input_tensor, FLAGS)
            with tf.name_scope('mix'):
                mixed = tf.matmul(input_tensor, weights)
                tf.summary.histogram('maybe_guassian', mixed)
            # Batch or layer normalization has to stay __after__ the affine
            # transformation (the bias term doens't really matter because of the
            # beta term in the normalization equation).
            # See pp. 5 of the batch normalization paper:
            # ```We add the BN transform immediately before the nonlinearity, by
            # normalizing x = W u + b```
            # biases = bias_variable('biases', [output_dim], layer_name)
            preactivate = self.normalize('layer_normalization', mixed, train_flag, FLAGS)  # + biases
            # tf.summary.histogram('pre_activations', preactivate)
            # preactivate = dropout_layer('dropout', preactivate)
            with tf.name_scope('activation') as scope:
                activations = self.activate(preactivate, funcname=act)
                tf.summary.histogram('activations', activations)
        return activations        
    
    def activate(self, input_layer, funcname=None):
        """Applies an activation function"""
        if isinstance(funcname, tuple):
            funcname = funcname[0]
            params = funcname[1:]
        if funcname is None:
            funcname = self.activation_func
        if funcname == 'LINEAR':
            return input_layer
        activation_map = {
            'IDENT':   tf.identity,
            'RELU':    tf.nn.relu,
            'RELU6':   tf.nn.relu6,
            'ELU':     tf.nn.elu,
            'SIGMOID': tf.nn.sigmoid,
            'TANH':    tf.nn.tanh,
            'LRELU':   lambda x, name: tf.maximum(params[0]*x, x, name=name)
        }
        return activation_map[funcname](input_layer, name=funcname.lower())
    
    def add_hidden_layers(self, features, architecture, FLAGS, train_flag, act=None):
        """Add hidden layers to the model using the architecture parameters."""
        hidden_out = features
        jit_scope = tf.contrib.compiler.jit.experimental_jit_scope #JIT compiler compiles and runs parts of TF graphs via XLA, fusing multiple operators (kernel fusion) nto a small number of compiled kernels.
        with jit_scope(): #this operation will be compiled with XLA.
            for hid_i in range(1, FLAGS.n_hidden + 1):
                hidden_out = self.nn_layer(hidden_out,
                                      architecture['n_hidden_{:1d}'.format(hid_i)],
                                      '{:1d}_hidden'.format(hid_i), FLAGS, act, train_flag)
        return hidden_out        
    

In [8]:
class FeedForward(object):
    
    def __init__(self, func, nstep_per_epoch=None, dtype='trainer'):
        
        if dtype == 'trainer':            
            self.nstep_per_epoch = nstep_per_epoch
            #self.architecture = architecture
            #self.FLAGS = FLAGS
            with tf.device('/cpu:0'):
                #self.global_step = tf.contrib.framework.get_or_create_global_step()
                # tf.train.get_global_step()
                self.global_step = tf.get_variable(
                    'global_step', [],
                    initializer=tf.constant_initializer(0),
                    dtype=tf.int64,
                    trainable=False)
        elif dtype != 'evaluator': #Evaluator
            raise ValueError('Invalid dtype value: ' + dtype)

        self.func = func
        self.dtype = dtype

    def get_learning_rate(self, initial_learning_rate):
        """Get the learning rate."""
        with tf.name_scope('learning_rate') as scope:
            if FLAGS.lr_decay_policy == 'poly':
                return tf.train.polynomial_decay(
                                        initial_learning_rate,
                                        self.global_step,
                                        decay_steps=FLAGS.epoch_num*self.nstep_per_epoch,
                                        end_learning_rate=0.,
                                        power=FLAGS.lr_poly_power,
                                        cycle=False)
            elif FLAGS.lr_decay_policy == 'exp':
                return tf.train.exponential_decay(
                                        initial_learning_rate,
                                        self.global_step,
                                        decay_steps=FLAGS.lr_decay_epochs*self.nstep_per_epoch,
                                        decay_rate=FLAGS.lr_decay_rate,
                                        staircase=True)
            else:            
                # decayed_lr = tf.train.exponential_decay(
                #     initial_learning_rate,
                #     global_step,
                #     FLAGS.decay_step,
                #     FLAGS.decay_base,
                #     staircase=False)
                decayed_lr = tf.train.inverse_time_decay(
                    initial_learning_rate,
                    self.global_step,
                    decay_steps=FLAGS.decay_step,
                    decay_rate=FLAGS.decay_rate)
                final_lr = tf.clip_by_value(
                    decayed_lr, FLAGS.rate_min, 1000, name=scope)
                tf.summary.scalar('value', final_lr)
                return final_lr
        # return self.learning_rate 

    def get_accuracy(self, labels_int, logits, name):
        """Get the accuracy tensor."""
        with tf.name_scope(name) as scope:
            # For a classifier model, we can use the in_top_k Op.
            # It returns a bool tensor with shape [batch_size] that is true for
            # the examples where the label is in the top k (here k=1)
            # of all logits for that example.
            correct = tf.nn.in_top_k(
                logits, labels_int, 1, name='correct_prediction') # returns a tensor of type bool.
            return tf.reduce_mean(tf.cast(correct, DT_FLOAT), name=scope)

    # auc = get_auc(labels, probs, True, 'metrics/auc')
    def get_auc(self, labels, scores, hist_flag, name):
        """Calculate the AUC of the two-way classifier for the given class."""

        def get_auc_using_histogram(labels, scores, class_, name):
            """Calculate the AUC."""
            class_ind = class_dict[class_.upper()]
            with tf.name_scope(name) as scope:
                auc, update_op = tf.contrib.metrics.auc_using_histogram( # his Op maintains Variables containing histograms of the scores associated with True and False labels. 
                    tf.cast(labels[:, class_ind], tf.bool),
                    scores[:, class_ind],
                    score_range=[0.0, 1.0],
                    nbins=200,
                    collections=None,
                    name=scope)
            ops.add_to_collections(ops.GraphKeys.UPDATE_OPS, update_op)
            # print(update_op.name)
            # print(auc) # it doesn't work because FailedPreconditionError (see above for traceback): Attempting to use uninitialized value metrics/auc/0//hist_accumulate/hist_true_acc
            # aucp = tf.Print(auc,[auc], message='AUC the label: ' + class_) # it doesnt work because it doesnt run in a session
            # print(aucp)
            return auc

        def get_auc_metric(labels, scores, class_, name):
            """Determine the AUC using conventional methods."""
            class_ind = class_dict[class_.upper()]
            with tf.name_scope(name) as scope:
                auc, _ = tf.metrics.auc( # Computes the approximate AUC via a Riemann sum.
                    tf.cast(labels[:, class_ind], tf.bool), # ?? Print out!!
                    scores[:, class_ind],
                    weights=None,
                    num_thresholds=200,
                    metrics_collections=None,
                    updates_collections=ops.GraphKeys.UPDATE_OPS,
                    curve='ROC',
                    name=scope)
            # print(auc.op.name)
            return auc

        classes = ['0', '3', '6', '9', 'C', 'F', 'R']
        class_dict = {classes[ind]: ind for ind in range(len(classes))}
        if hist_flag:
            auc_func = get_auc_using_histogram
        else:
            auc_func = get_auc_metric
        with tf.name_scope(name) as scope:
            aucv = [
                    auc_func(labels, scores, class_, str(ind)) for ind, class_ in enumerate(classes) # pair (index ej. 0, value ej. '0')
                   ]      
            auc_values = tf.stack( # Pack along first dim
                aucv,
                axis=0,
                name=scope)
            # aucv = tf.Print(auc_values,[auc_values], message='AUC for all labels: ')
            # print(aucv) # or maybe aucv.eval() or var = tf.Variable(aucv) and then var.eval(session=sess), or ovar = sess.run(var) but Attempting to use uninitialized value metrics/auc/Variable
            return auc_values


    # conf_mtx = get_confusion_matrix(labels_int, predictions, len(classes), 'metrics/confusion')
    def get_confusion_matrix(self, labels_int, predictions, num_classes, name):
        """Get the confusion matrix.
        Both prediction and labels must be 1-D arrays of the same shape in order for 
        this function to work.
        """
        with tf.name_scope(name) as scope:
            conf = tf.confusion_matrix(
                labels_int,
                predictions=predictions,
                num_classes=num_classes,
                dtype=tf.int32,
                name=scope,
                weights=None)
        # print(conf.op.name)
        return conf #return a K x K Matriz K = num_classes


    def get_m_hand(self, labels, scores, name):
        """Implement the M measure described in Hand.

        See ```A Simple Generalisation of the Area Under the ROC Curve for Multiple
        Class Classification Problems``` Hand, Till 2001.    

        """
        def get_auc_using_histogram(labels, scores, first_ind, second_ind, scope):
            """Calculate the AUC.
            Calculate the AUC value by maintainig histograms of boolean variables (labels and 
            scores masked by the First-Second Individuals rule).
            """
            mask = (labels[:, first_ind] + labels[:, second_ind]) > 0 #one in at least one column.
            auc, update_op = tf.contrib.metrics.auc_using_histogram( # maintains variables containing histograms of the scores associated with True, False labels. 
                tf.cast(tf.boolean_mask(labels[:, first_ind], mask), tf.bool), # tf.boolean_mask: Apply boolean mask to tensor. Numpy equivalent is tensor[mask].
                tf.boolean_mask(scores[:, first_ind], mask),
                score_range=[0.0, 1.0],
                nbins=500,
                collections=None,
                name=scope)
            ops.add_to_collections(ops.GraphKeys.UPDATE_OPS, update_op)
            # print(update_op.name)
            return auc

        temp_array = []
        with tf.name_scope(name) as main_scope:
            for first_ind in range(7):
                for second_ind in range(7):
                    if first_ind != second_ind:
                        final_name = '{:d}{:d}'.format(first_ind, second_ind)
                        with tf.name_scope(final_name) as scope:
                            auc = get_auc_using_histogram(
                                labels, scores, first_ind, second_ind, scope)
                        temp_array.append(auc)
            return tf.stack(temp_array, axis=0, name=main_scope) # Stacks a list of rank-R tensors into one rank-(R+1) tensor.


    def get_auc_pr_curve(self, labels, scores, name, num_thresholds):    
        with tf.name_scope(name) as scope:                             
            AUC_PR = []
            AUC_data = []
            for i in range(7):  
                data, update_op = tf.contrib.metrics.precision_recall_at_equal_thresholds(
                                name='pr_data',
                                predictions=scores[:, i],
                                labels=tf.cast(labels[:, i], tf.bool),
                                num_thresholds=10, use_locking=True)
                ops.add_to_collections(ops.GraphKeys.UPDATE_OPS, update_op)
                AUC_data.append((tf.stack(data.recall), tf.stack(data.precision), tf.stack(data.thresholds)))   # we cant use sklearn with tensorflow definition!
                auc, _ = tf.metrics.auc(labels[:, i], scores[:, i], weights=None, num_thresholds=10, 
                                        curve='PR', updates_collections=ops.GraphKeys.UPDATE_OPS, metrics_collections=None, summation_method='careful_interpolation') # 
                # ops.add_to_collections(ops.GraphKeys.UPDATE_OPS, update_op)
                AUC_PR.append(auc)
            # print(AUC_data)
            return tf.stack( # Pack the array of scalar tensor along one dim tensor
                AUC_PR,
                axis=0,
                name=scope), AUC_data

    def log_loss(self, labels, probs, name):
        """
        Args:
            labels: Labels tensor, int32 - [batch_size, n_classes], with one-hot
            encoded values.
            logits: Probabilities tensor, float32 - [batch_size, n_classes].
        """
        with tf.name_scope(name) as scope:
            total_loss = 0
            for j in range(probs.shape[1].value):
                loss = tf.losses.log_loss(labels[:, j], probs[:, j], loss_collection=None)
                total_loss += loss

            return tf.div(total_loss, np.float32(probs.shape[1].value), name=scope)
    
    def calculate_metrics(self, labels, logits):
        """Evaluate the quality of the logits at predicting the label.

        Args:
            labels: Labels tensor, int32 - [batch_size, n_classes], with one-hot
            encoded values.
            logits: Logits tensor, float32 - [batch_size, n_classes].
        Returns:
            A scalar float32 tensor with the fraction of examples (out of
            batch_size) that were predicted correctly.
        """
        classes = ['0', '3', '6', '9', 'C', 'F', 'R']
        with tf.name_scope('metrics'):
            labels_int = tf.argmax(labels, 1, name='intlabels') #tf.argmax: Returns the index with the largest value across axes=1 of a tensor.		
            predictions = tf.argmax(logits, 1, name='predictions')        
            probs = tf.nn.softmax(logits, name='probs') # Computes softmax activations. softmax = tf.exp(logits) / tf.reduce_sum(tf.exp(logits), axis)        

        m_list = self.get_m_hand(labels, probs, 'metrics/m_measure')
        accuracy = self.get_accuracy(labels_int, logits, 'metrics/accuracy')    
        auc = self.get_auc(labels, probs, True, 'metrics/auc')    
        conf_mtx = self.get_confusion_matrix(labels_int, predictions,
                                        len(classes), 'metrics/confusion')
        lloss = self.log_loss(labels, probs, 'metrics/log_loss')
        pr_auc, pr_data = self.get_auc_pr_curve(labels, probs, 'metrics/auc_pr', 200)

        # this is for the definition of the graph:
        return accuracy, conf_mtx, auc, m_list, lloss, pr_auc, pr_data
    
    def training_step(self, architecture, FLAGS):        
        features = tf.placeholder(
            DT_FLOAT, [None, architecture['n_input']], name='features')
        labels = tf.placeholder(
            DT_FLOAT, [None, architecture['n_classes']], name='targets')
        # epoch_flag = tf.placeholder(tf.int32, None, name='epoch_flag')
        example_weights = tf.placeholder(
            DT_FLOAT, [None], name='example_weights')
        
        with tf.device('/gpu:0'):
            # Evaluate the loss:
            loss, logits = self.func(features, labels, example_weights, architecture, FLAGS)

        with tf.device('/cpu:0'): # No in_top_k implem on GPU
            accuracy, conf_mtx, auc_list, m_list, lloss, auc_pr, auc_data = self.calculate_metrics(labels, logits)
            better_acc = tf.reduce_mean(tf.diag_part(conf_mtx / tf.reduce_sum(conf_mtx, axis=1, keepdims=True))) #recall
            precision = tf.reduce_mean(tf.diag_part(conf_mtx / tf.reduce_sum(conf_mtx, axis=0, keepdims=True)))            
            auc_mean = tf.reduce_mean(auc_list)
            m_list_mean = tf.reduce_mean(m_list)
            auc_pr_mean = tf.reduce_mean(auc_pr)
            
            with tf.name_scope('0_performance'):
                # Scalar summaries to track the loss and accuracy over time in TB.
                tf.summary.scalar('0accuracy', accuracy)
                tf.summary.scalar('1better_accuracy', better_acc)
                tf.summary.scalar('12precision', precision)
                # tf.summary.scalar('f1score', f1score)
                tf.summary.scalar('2auc_aoc', auc_mean)
                tf.summary.scalar('3m_measure', m_list_mean)
                tf.summary.scalar('4loss', loss)
                tf.summary.scalar('5log_loss', lloss)
                tf.summary.scalar('6auc_pr', auc_pr_mean)

        # Apply the gradients to optimize the loss function
        with tf.device('/gpu:0'):            
            update_ops = ops.get_collection(ops.GraphKeys.UPDATE_OPS)
            # print(update_ops)
            with ops.control_dependencies(update_ops):
                with tf.name_scope('train') as scope:
                    # print_loss = tf.Print(loss, [loss], name='print_loss') 

                    # Create a variable to track the global step.
        #            global_step = tf.get_variable(
        #                'train/global_step',
        #                shape=[],
        #                initializer=tf.constant_initializer(0, dtype=tf.int32),
        #                trainable=False)            
                    # Horovod: adjust learning rate based on number of GPUs.
                    # optimizer = tf.train.GradientDescentOptimizer(1.0 * hvd.size())
                    final_learning_rate = self.get_learning_rate(FLAGS.learning_rate * hvd.size())

                    # optimizer = tf.train.GradientDescentOptimizer(final_learning_rate)
                    optimizer = tf.train.MomentumOptimizer(final_learning_rate, FLAGS.momentum, use_nesterov=True)
                    # optimizer = tf.train.AdagradOptimizer(final_learning_rate)

                    # Use the optimizer to apply the gradients that minimize the loss
                    # (and increment the global step counter) as a single training step.
        #            return optimizer.minimize(
        #                loss, global_step=global_step, name=scope)
                    optimizer = hvd.DistributedOptimizer(optimizer) #HVD!!
                    train_op = optimizer.minimize(loss, global_step=self.global_step, name=scope)
            
                        
        return train_op, final_learning_rate, conf_mtx, accuracy, better_acc, precision, auc_list, auc_mean, m_list, m_list_mean, loss, lloss, auc_pr, auc_pr_mean, auc_data

    def evaluation_step(self, batch_size):
        
        if (self.dtype!='evaluator'):
            raise ValueError('Invalid function for dtype: ' + self.dtype)
            
        features = tf.placeholder(
            DT_FLOAT, [None, architecture['n_input']], name='features')
        labels = tf.placeholder(
            DT_FLOAT, [None, architecture['n_classes']], name='targets')
        example_weights = tf.placeholder(
            DT_FLOAT, [None], name='example_weights')

        with tf.device('/gpu:0'):
            # Evaluate the loss:
            logits = self.func(features, labels, architecture, FLAGS)        
            accuracy, conf_mtx = self.calculate_metrics(labels, logits)[:2]
            better_acc = tf.reduce_mean(tf.diag_part(conf_mtx / tf.reduce_sum(conf_mtx, axis=1, keepdims=True)))
            precision = tf.reduce_mean(tf.diag_part(conf_mtx / tf.reduce_sum(conf_mtx, axis=0, keepdims=True)))
            #f1score = 2 * precision * better_acc / (precision + better_acc)
            

        return conf_mtx, accuracy, better_acc, precision #,f1score  #, auc_list, m_list, lloss, auc_pr, auc_data
    
    def init(self):
        # init_op = tf.global_variables_initializer()
        # sess.run(init_op)        
        """Add an Op to the graph to initialize the global and local variables."""
        with tf.name_scope('init') as scope:
            with tf.name_scope('global'):
                global_init = tf.global_variables_initializer()
            with tf.name_scope('local'):
                local_init = tf.local_variables_initializer()
                # print(local_init.name)
            #init_op = tf.group(global_init, local_init, name=scope)
        return global_init, local_init
        
    def sync(self, sess):
        sync_op = hvd.broadcast_global_variables(0)
        sess.run(sync_op)

In [9]:
def loss_func(features, labels, weights, architecture, FLAGS):
    # Build the forward model
    net = GPUNetworkBuilder(dtype=DT_FLOAT)
    train_flag = tf.placeholder(tf.bool, None, name='train_flag')
    with tf.name_scope('input_normalization') as scope:
        feature_norm = features
        net.variable_summaries('input_normalized', feature_norm, FLAGS.allow_summaries)
    hidden_out = net.add_hidden_layers(feature_norm, architecture, FLAGS, train_flag)
    # Linear output layer for the logits
    logits = (net.nn_layer(hidden_out, architecture['n_classes'],'9_softmax_linear', FLAGS, 'IDENT', train_flag))
    
    with tf.name_scope('loss') as scope:
        with tf.name_scope('regularization'):
            penalty = tf.losses.get_regularization_loss(name='penalty') #Gets the total regularization loss from an optional scope name (sum for ol + 3h + 2h + 1h).
            tf.summary.scalar('weight_norm', penalty / (1e-8 + FLAGS.reg_rate)) #for printing out
        with tf.name_scope('cross_entropy') as xentropy_scope:
            weighted_cross_entropy = tf.losses.softmax_cross_entropy(
                onehot_labels=labels,
                logits=logits,
                weights=weights,  # 1.0,
                scope=xentropy_scope,
                loss_collection=ops.GraphKeys.LOSSES)
            tf.summary.scalar('weighted_cross_entropy', weighted_cross_entropy)
        loss= tf.add(weighted_cross_entropy, penalty, name=scope) # Returns x + y element-wise.    
            
    return loss, logits

In [10]:
def eval_func(features, labels, architecture, FLAGS):
    # Build the forward model
    FLAGS.allow_summaries = False
    net = GPUNetworkBuilder(dtype=DT_FLOAT)
    train_flag = tf.placeholder(tf.bool, None, name='train_flag')    
    hidden_out = net.add_hidden_layers(features, architecture, FLAGS, train_flag)
    # Linear output layer for the logits
    logits = (net.nn_layer(hidden_out, architecture['n_classes'],'9_softmax_linear', FLAGS, 'IDENT', train_flag))
                
    return logits


## Main 

In [11]:
global_start_time = time.time()
tf.set_random_seed(1234+hvd.rank())
np.random.seed(4321+hvd.rank())

# create logger:
log_name = FLAGS.name + '_' + str(hvd.rank())
logger = logging.getLogger(log_name)
logger.setLevel(logging.DEBUG)  # INFO, ERROR
# file handler which logs debug messages
fh = logging.FileHandler(os.path.join(FLAGS.logdir, log_name + '.log'))
fh.setLevel(logging.DEBUG)
# console handler
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
# add formatter to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
# add handlers to logger
logger.addHandler(fh)
logger.addHandler(ch)
print(logger)

<Logger Xworkers_1mill_0 (DEBUG)>


#### Download the data if it has not been downloaded

In [12]:
def download_data(rank, FLAGS):
    server = ftplib.FTP()
    server.connect(str(os.environ.get("FTP_HOST")), int(os.environ.get("FTP_PORT")))
    server.login(os.environ.get("FTP_USER"), os.environ.get("FTP_PASS"))

    server.cwd(FLAGS.ftp_dir)               # change into ftp_dir directory
    logger.info("FTP connection stablished by worker:  {}".format(rank))
    
    filenames = server.nlst() # get filenames within the directory
        
    train_suffix = 'train_%d.h5' % rank
    if (rank==0):
        if FLAGS.eval:        
            fname_suffix = 'test_%d.h5' % rank
            filenames = [elem for elem in filenames if fname_suffix in elem]        
        else:
            valid_suffix = 'valid_%d.h5' % rank
            filenames = [elem for elem in filenames if (train_suffix in elem or valid_suffix in elem)]                
    else:
        filenames = [elem for elem in filenames if (train_suffix in elem)]

    for filename in filenames:        
        if FLAGS.eval:
            local_path = os.path.join(PRO_DIR, FLAGS.test_dir, filename)    
        else:
            if (str('train') in filename[-10:-5]):
                local_path = os.path.join(PRO_DIR, FLAGS.train_dir, filename)    
            elif (str('valid') in filename[-10:-5]):
                local_path = os.path.join(PRO_DIR, FLAGS.valid_dir, filename)   
            else: 
                continue
                
        if not os.path.exists(local_path):            
            file = open(local_path, 'wb')
            server.retrbinary('RETR '+ filename, file.write, 8*1024)            
            file.close()
            logger.info("file downloaded:  {}".format(filename))

    server.quit() # This is the “polite” way to close a connection
    logger.info("FTP connection closed by worker:  {}".format(rank))

def download_data_by_rank(rank, FLAGS):
    server = ftplib.FTP()
    server.connect(str(os.environ.get("FTP_HOST")), int(os.environ.get("FTP_PORT")))
    server.login(os.environ.get("FTP_USER"), os.environ.get("FTP_PASS"))

    server.cwd(FLAGS.ftp_dir)               # change into ftp_dir directory
    logger.info("FTP connection stablished by worker:  {}".format(rank))
    
    filenames = server.nlst() # get filenames within the directory
    fname_suffix = '_%d.h5' % rank
    filenames = [elem for elem in filenames if fname_suffix in elem]        
    
    for filename in filenames:            
        local_path = os.path.join(PRO_DIR, FLAGS.train_dir, filename)                    
        if not os.path.exists(local_path):            
            file = open(local_path, 'wb')
            server.retrbinary('RETR '+ filename, file.write, 8*1024)            
            file.close()
            logger.info("file downloaded:  {}".format(filename))

    server.quit() 
    logger.info("FTP connection closed by worker:  {}".format(rank))


#download_data_by_rank(hvd.rank(), FLAGS)
download_data(hvd.rank(), FLAGS)

2018-11-20 18:20:28,517 - Xworkers_1mill_0 - INFO - FTP connection stablished by worker:  0
2018-11-20 18:20:28,517 - Xworkers_1mill_0 - INFO - FTP connection stablished by worker:  0
2018-11-20 18:21:02,113 - Xworkers_1mill_0 - INFO - file downloaded:  temporalloandynmodifmrstaticitur15mill-16mill-allncols-train_0.h5
2018-11-20 18:21:02,113 - Xworkers_1mill_0 - INFO - file downloaded:  temporalloandynmodifmrstaticitur15mill-16mill-allncols-train_0.h5
2018-11-20 18:21:06,808 - Xworkers_1mill_0 - INFO - file downloaded:  temporalloandynmodifmrstaticitur15mill-16mill-allncols-valid_0.h5
2018-11-20 18:21:06,808 - Xworkers_1mill_0 - INFO - file downloaded:  temporalloandynmodifmrstaticitur15mill-16mill-allncols-valid_0.h5
2018-11-20 18:21:06,853 - Xworkers_1mill_0 - INFO - FTP connection closed by worker:  0
2018-11-20 18:21:06,853 - Xworkers_1mill_0 - INFO - FTP connection closed by worker:  0


In [13]:
def get_num_records(tf_record_pattern):
    def count_records(file_name):
        count = 0
        for _ in tf.python_io.tf_record_iterator(tf_record_filename):
            count += 1
        return count
    filenames = sorted(tf.gfile.Glob(tf_record_pattern))
    nfile = len(filenames)
    return (count_records(filenames[0])*(nfile-1) +
            count_records(filenames[-1]))

def get_files_dict(FLAGS):        
    ext = "*.h5"

    if (hvd.rank()==0):
        files_dict = {'train': glob.glob(os.path.join(PRO_DIR, FLAGS.train_dir, ext)), 
                      'valid': glob.glob(os.path.join(PRO_DIR, FLAGS.valid_dir, ext)), 
                      'test': glob.glob(os.path.join(PRO_DIR, FLAGS.test_dir, ext))}
    else:
        files_dict = {'train': glob.glob(os.path.join(PRO_DIR, FLAGS.train_dir, ext))}

    return files_dict

def architecture_settings(files_dict, FLAGS):
    architecture = {}
    architecture['rank'] = hvd.rank()
    ok_inputs = True
    for key in files_dict.keys():
        total_records = 0
        for file in files_dict[key]:                                
            with pd.HDFStore(file) as dataset_file:
                if (ok_inputs): 
                    index_length = len(dataset_file.get_storer(key+'/features').attrs.data_columns)
                    architecture['n_input'] = dataset_file.get_storer(key+ '/features').ncols - index_length
                    architecture['n_classes'] = dataset_file.get_storer(key+'/labels').ncols - index_length
                    ok_inputs = False                
                total_records += dataset_file.get_storer(key + '/features').nrows
        architecture[key + '_num_examples'] = total_records                            
    
    if FLAGS.eval:
        architecture['total_num_examples'] = architecture['test_num_examples']
    else:
        if FLAGS.total_examples == -1:
            architecture['total_num_examples'] = architecture['train_num_examples']
        else:
            architecture['total_num_examples'] = FLAGS.total_examples 
    
    for hid_i in range(1, FLAGS.n_hidden+1):
        architecture['n_hidden_{:1d}'.format(hid_i)] = FLAGS.s_hidden[hid_i-1]
    # print('rank: ', hvd.rank(), 'architecture', architecture)   
    # time.sleep(5)
    return architecture

In [14]:
#To sum up the dataset per worker (assuming the same size of files per worker approximately):
files_dict = get_files_dict(FLAGS)
architecture = architecture_settings(files_dict, FLAGS)

nrecord = architecture['total_num_examples']

In [15]:
logger.info("Num ranks:  {}".format(hvd.size()))
logger.info("Num of records: {}".format(nrecord))
logger.info("Total batch size: {}".format(FLAGS.batch_size * hvd.size()))
logger.info("{}, per device".format(FLAGS.batch_size))
logger.info("Data type: {}".format(DT_FLOAT)) 
logger.info("architecture: {}".format(architecture)) 
# time.sleep(5)

2018-11-20 18:21:36,778 - Xworkers_1mill_0 - INFO - Num ranks:  1
2018-11-20 18:21:36,778 - Xworkers_1mill_0 - INFO - Num ranks:  1
2018-11-20 18:21:36,780 - Xworkers_1mill_0 - INFO - Num of records: 52000000
2018-11-20 18:21:36,780 - Xworkers_1mill_0 - INFO - Num of records: 52000000
2018-11-20 18:21:36,781 - Xworkers_1mill_0 - INFO - Total batch size: 8850
2018-11-20 18:21:36,781 - Xworkers_1mill_0 - INFO - Total batch size: 8850
2018-11-20 18:21:36,783 - Xworkers_1mill_0 - INFO - 8850, per device
2018-11-20 18:21:36,783 - Xworkers_1mill_0 - INFO - 8850, per device
2018-11-20 18:21:36,785 - Xworkers_1mill_0 - INFO - Data type: <dtype: 'float32'>
2018-11-20 18:21:36,785 - Xworkers_1mill_0 - INFO - Data type: <dtype: 'float32'>
2018-11-20 18:21:36,786 - Xworkers_1mill_0 - INFO - architecture: {'rank': 0, 'n_input': 258, 'n_classes': 7, 'train_num_examples': 26769588, 'valid_num_examples': 901868, 'test_num_examples': 0, 'total_num_examples': 52000000, 'n_hidden_1': 200, 'n_hidden_2': 1

#### TF Operations

In [16]:
if FLAGS.eval:
    if FLAGS.test_dir is None:
        logger.error("eval requires data_dir to be specified")
        raise ValueError("eval requires data_dir to be specified")
    if hvd.size() > 1:
        logger.error("Multi-GPU evaluation is not supported")
        raise ValueError("Multi-GPU evaluation is not supported")
    evaluator = FeedForward(eval_func, dtype='evaluator')
    logger.info("Building evaluation graph")
    conf_mtx_op, accuracy_op, better_acc_op, precision_op = evaluator.evaluation_step(FLAGS.test_batch_size)    
    print(evaluator)
else:    
    nstep_per_epoch = nrecord // FLAGS.batch_size # if it is kwnow the total size: (FLAGS.batch_size * hvd.size())
    logger.info("Number of steps per epoch: %d" % nstep_per_epoch)
    # model_func = lambda features, labels, architecture, FLAGS: loss_func(features, labels, architecture, FLAGS) # inference_vgg(net, images, nlayer)
    trainer = FeedForward(loss_func, nstep_per_epoch=nstep_per_epoch)
    logger.info("Building training graph")    
    train_ops, learning_rate_op, conf_mtx_op, accuracy_op, better_acc_op, precision_op, auc_list_op, auc_mean_op, m_list_op, m_list_mean_op, total_loss_op, lloss_op, auc_pr_op, auc_pr_mean_op, auc_data_op = trainer.training_step(architecture, FLAGS)
    logger.info("Graph building completed....")
    print(trainer)
    global_init, local_init = trainer.init()

logger.info("Creating session")
config = tf.ConfigProto(allow_soft_placement = True)
config.intra_op_parallelism_threads = 1
config.inter_op_parallelism_threads = 10
config.gpu_options.force_gpu_compatible = True
config.gpu_options.visible_device_list = str(hvd.local_rank())

2018-11-20 18:21:37,881 - Xworkers_1mill_0 - INFO - Number of steps per epoch: 5875
2018-11-20 18:21:37,881 - Xworkers_1mill_0 - INFO - Number of steps per epoch: 5875
2018-11-20 18:21:37,888 - Xworkers_1mill_0 - INFO - Building training graph
2018-11-20 18:21:37,888 - Xworkers_1mill_0 - INFO - Building training graph


Instructions for updating:
Please switch to remove_squeezable_dimensions from tf.confusion_matrix. Note that the order of the inputs and outputs of labels and predictions have also been switched.


2018-11-20 18:21:42,749 - tensorflow - WARNING - From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/contrib/metrics/python/ops/histogram_ops.py:83: remove_squeezable_dimensions (from tensorflow.contrib.framework.python.framework.tensor_util) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to remove_squeezable_dimensions from tf.confusion_matrix. Note that the order of the inputs and outputs of labels and predictions have also been switched.
2018-11-20 18:21:55,530 - Xworkers_1mill_0 - INFO - Graph building completed....
2018-11-20 18:21:55,530 - Xworkers_1mill_0 - INFO - Graph building completed....
2018-11-20 18:21:55,537 - Xworkers_1mill_0 - INFO - Creating session
2018-11-20 18:21:55,537 - Xworkers_1mill_0 - INFO - Creating session


In [17]:
#print(FLAGS.logdir)

#### Defining summary (writer) and checkpoint (saver) files

In [18]:
sess = tf.Session(config=config)

train_writer = None
valid_writer = None
saver = None
summary_ops = None


if hvd.rank() == 0 and FLAGS.logdir is not None:
    train_writer = tf.summary.FileWriter(os.path.join(FLAGS.logdir), sess.graph)
    valid_writer = tf.summary.FileWriter(os.path.join(FLAGS.logdir, 'valid'), graph=None)
    summary_ops = tf.summary.merge_all()
    last_summary_time = time.time()
    saver = tf.train.Saver(keep_checkpoint_every_n_hours=1)
    last_save_time = time.time()

if not FLAGS.eval:        
    logger.info("Initializing variables")    
    sess.run([global_init, local_init])

restored = False
if hvd.rank() == 0 and saver is not None:
    ckpt = tf.train.get_checkpoint_state(FLAGS.logdir)
    checkpoint_file = os.path.join(FLAGS.logdir, "checkpoint")
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        restored = True
        logger.info("Restored session from checkpoint {}".format(ckpt.model_checkpoint_path))
    else:
        if not os.path.exists(FLAGS.logdir):
            os.mkdir(FLAGS.logdir)

2018-11-20 18:21:58,679 - Xworkers_1mill_0 - INFO - Initializing variables
2018-11-20 18:21:58,679 - Xworkers_1mill_0 - INFO - Initializing variables


#### Running evaluation from a checkpoint file

In [19]:
def calculate_better_acc(conf_mtx, axis=1):
    cfsum = conf_mtx.sum(axis=axis, keepdims=True)
    conf_mtx1 = np.divide(conf_mtx, cfsum, out=np.zeros_like(conf_mtx, dtype=np.float32), where=(cfsum!=0), dtype=np.float32)    
    bett_acc = conf_mtx1.diagonal().mean()
    return bett_acc

#### Getting Dataset

In [20]:
if not FLAGS.eval:

    if (hvd.rank()==0):
        DATA = md.get_h5_data(PRO_DIR, architecture, FLAGS.train_dir, FLAGS.valid_dir, None, train_period=FLAGS.train_period, valid_period=FLAGS.valid_period, test_period=FLAGS.test_period)         
    else:
        DATA = md.get_h5_data(PRO_DIR, architecture, FLAGS.train_dir, None, None, train_period=FLAGS.train_period, valid_period=FLAGS.valid_period, test_period=FLAGS.test_period) 
        
    #logger.info('Features List: {}'.format(DATA.train.features_list))
    #logger.info('Labels List: {}'.format(DATA.train.labels_list))
    
else:
    DATA = md.get_h5_data(PRO_DIR, architecture, None, None, FLAGS.test_dir, train_period=FLAGS.train_period, valid_period=FLAGS.valid_period, test_period=FLAGS.test_period) 
    #logger.info('Features List: {}'.format(DATA.test.features_list))
    #logger.info('Labels List: {}'.format(DATA.test.labels_list))


/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_train/temporalloandynmodifmrstaticitur15mill-16mill-allncols-train_0.h5 ...to load
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_train/temporalloandynmodifmrstaticitur15mill-16mill-allncols-train_0.h5  loaded in RAM
class_weights [  445264   736172   259623   614496 24186991   499157    27885]
class_weights [0.983 0.972 0.99  0.977 0.096 0.981 0.999]
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_valid/temporalloandynmodifmrstaticitur15mill-16mill-allncols-valid_0.h5 ...to load
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_valid/temporalloandynmodifmrstaticitur15mill-16mill-allncols-valid_0.h5  loaded in RAM
class_weights [ 14560  19805   6681  15002 835450   9960    410]
class_weights [0.984 0.978 0.993 0.983 0.074 0.989 1.   ]


In [21]:
print(DATA.train._dict[0]['nrows'])
print(nrecord)

26769588
52000000


In [22]:
def create_feed_dict(tag, DATA, FLAGS):
    """Create the feed dictionary for mapping data onto placeholders in the graph."""
    if tag == 'batch':
        if hvd.rank()==0:
            batch_size = FLAGS.batch_size // 2
        else:
            batch_size = FLAGS.batch_size
        
        features, targets = DATA.train.next_random_batch(batch_size)        
        #example_weights = np.sum(np.multiply(np.float32(oh_labels),class_weights4), axis=1)
        example_weights = np.sum(np.multiply(np.float32(targets),DATA.train.class_weights), axis=1)
        #print('example_weights', example_weights)
        #example_weights = np.array([1.0], dtype=NP_FLOAT) # [1.0] * architecture['total_num_examples']                    
        # logger.info('features shape: {}'.format(features.shape))
    elif tag == 'train':
        features = DATA.train.orig.features
        targets = DATA.train.orig.labels
        example_weights = np.ones_like(targets.iloc[:, 1].values)
    elif tag == 'valid':
        features, targets, example_weights = DATA.validation.next_sequential_batch(FLAGS.valid_batch_size)
    else:
        features, targets, example_weights = DATA.test.next_sequential_batch(FLAGS.test_batch_size)

    # features[:, :7] = targets
    if tag == 'batch':
        k_prob_input = 0.9  # 0.9  # .85  # .75  # 0.8  # 0.6
        k_prob = FLAGS.dropout_keep
        t_flag = True
    else:
        k_prob_input = 1.0
        k_prob = 1.0
        t_flag = False

    # Change the python dictionary to an io-buffer for a better performance.
    # See here:
    # https://www.tensorflow.org/performance/performance_guide
    feed_d = {
        'features:0': features,
        'targets:0': targets,
        'example_weights:0': example_weights,        
        'train_flag:0': t_flag,
        #'epoch_flag:0': FLAGS.epoch_flag,
        #'1_hidden/dropout/keep_proba:0': k_prob_input,
        #'2_hidden/dropout/keep_proba:0': k_prob,
        # '3_hidden/dropout/keep_proba:0': k_prob,
        # '4_hidden/dropout/keep_proba:0': k_prob,
        # '5_hidden/dropout/keep_proba:0': k_prob,
        '9_softmax_linear/dropout/keep_proba:0': k_prob
    }
	
	# for any tag:
    if (FLAGS.n_hidden > 0) :
        # print ('k_prob_input', k_prob_input, type(k_prob_input))
        feed_d['1_hidden/dropout/keep_proba:0'] = k_prob_input
        for hid_i in range(2, FLAGS.n_hidden+1):
            feed_d['{:1d}_hidden/dropout/keep_proba:0'.format(hid_i)] = k_prob
    # print('feed_d', feed_d)
    # print('batch shape: ', features.shape)    
    return feed_d

In [23]:
def run_evaluation(nstep, sess, enqueue_ops):
    logger.info("Evaluating Model")
    accuracys = []
    better_accs = []
    logger.info("  Step  Accuracy  Better-Accuracy   Precision   f1score_micro   f1score_macro")
    for step in range(nstep):
        try:
            feed = create_feed_dict('test', DATA, FLAGS)     
            #logger.info('feed dictionary was created')
            conf_mtx, accuracy, better_acc, precision = sess.run(enqueue_ops, feed_dict=feed)
            #logger.info('operations were ran')
            if (math.isnan(better_acc)):                
                better_acc = calculate_better_acc(conf_mtx)                    
                #print('beter_acc nan')
            if (math.isnan(precision)):                
                precision = calculate_better_acc(conf_mtx, axis=0)                    
                #print('precision nan')
            f1score_micro = 2 * precision * better_acc / (precision + better_acc)
            f1score_macro = calculate_macro_f1score(np.array(conf_mtx, dtype='float32'))
            
            #if step == 0 or (step+1) % FLAGS.display_every == 0:
            logger.info("% 6i %5.1f%% %5.1f%% %5.1f%% %5.5f%%  %5.5f%%" % (step+1, accuracy*100, better_acc*100, precision*100, f1score_micro, f1score_macro))
            accuracys.append(accuracy)
            better_accs.append(better_acc)
            #print('acc and bett_acc appended')
            
        except KeyboardInterrupt:
            print("Keyboard interrupt")
            break
        except  Exception  as e:        
            raise ValueError('Error running Evaluation: ' + str(e))    

    nstep = len(accuracys)
    if nstep == 0:
        return
    accuracys = np.asarray(accuracys) * 100.
    better_accs = np.asarray(better_accs) * 100.
    acc_mean = np.mean(accuracys)
    bettacc_mean = np.mean(better_accs)
    if nstep > 2:
        acc_uncertainty = np.std(accuracys, ddof=1) / np.sqrt(float(nstep))
        bettacc_uncertainty = np.std(better_accs, ddof=1) / np.sqrt(float(nstep))
    else:
        acc_uncertainty = float('nan')
        bettacc_uncertainty = float('nan')
    acc_madstd = 1.4826*np.median(np.abs(accuracys - acc_mean))
    bettacc_madstd = 1.4826*np.median(np.abs(better_accs - bettacc_mean))
    logger.info('-' * 64)
    logger.info('Validation Accuracy: %.3f %% +/- %.2f (jitter = %.1f)' % (
        acc_mean, acc_uncertainty, acc_madstd))
    logger.info('Validation Better Accuracy: %.3f %% +/- %.2f (jitter = %.1f)' % (
        bettacc_mean, bettacc_uncertainty, bettacc_madstd))
    logger.info('-' * 64)    

In [24]:
nstep = 0
if FLAGS.eval:
    if not restored:
        logger.error("No checkpoint found for evaluation")
        raise ValueError("No checkpoint found for evaluation")
    else:        
        #nstep = nrecord // FLAGS.test_batch_size 
        nstep = DATA.test.total_num_batch(FLAGS.test_batch_size) 
        logger.info("total steps: {}".format(nstep))        
        enq_ops = [conf_mtx_op, accuracy_op, better_acc_op, precision_op]
        logger.info("Executing Evaluation")        
        run_evaluation(nstep, sess, enq_ops)   
        logger.info('Evaluation was done')
        # sys.exit(0) #the following instructiones will not be  executed
        quit()
else:    
    if FLAGS.epoch_num is not None:
        if (nrecord <= 0):
            logger.error("num_epochs requires nrecord to be specified")
            raise ValueError("num_epochs requires nrecord to be specified")
        nstep = math.ceil(np.float32(nrecord * FLAGS.epoch_num / FLAGS.batch_size)) # if it is kwnow the total size: (FLAGS.batch_size * hvd.size()) 

In [25]:
logger.info('METRICS:  %s\r\n' % str(FLAGS))
logger.info('Number of total steps: %d' % nstep)

2018-11-20 18:23:59,672 - Xworkers_1mill_0 - INFO - METRICS:  Namespace(allow_summaries=True, batch_layer_type='batch', batch_norm=True, batch_size=8850, batch_type='layer', data_dir='/input_data', decay_rate=1, decay_step=3520000, display_every=100, dropout=True, dropout_keep=0.9, epoch_flag=0, epoch_num=15, eval=False, ftp_dir='processed/c1mill', learning_rate=0.1, log_file=None, logdir=PosixPath('/home/ubuntu/summ_15ep_2wrk'), loss_tolerance=0.0001, lr_decay_epochs=30, lr_decay_policy='time', lr_decay_rate=0.1, lr_poly_power=2.0, max_epoch_size=-1, momentum=0.5, n_hidden=3, name='Xworkers_1mill', nstep_burnin=20, rate_min=0.0015, reg_rate=1e-05, s_hidden=[200, 140, 140], save_interval=450, stratified_flag=False, summary_interval=1800, test_batch_size=1200000, test_dir='chuncks_random_c1millx2_test', test_flag=True, test_period=[330, 342], total_examples=52000000, train_dir='chuncks_random_c1millx2_train', train_period=[121, 323], valid_batch_size=1200000, valid_dir='chuncks_random_c

In [26]:
# broadcast_global_variables from hvd
trainer.sync(sess)

#### Running Training 

In [27]:
# Trying to restore for training:
if hvd.rank() == 0 and not restored:
    if saver is not None:
        save_path = saver.save(sess, checkpoint_file, global_step=0)
        print("Checkpoint written to", save_path)

logger.info("Writing summaries to {}".format(FLAGS.logdir))
logger.info("Training")
logger.info("  Step; Epoch; time-per-record(sec);  batchtime/worker(sec);  Loss;   Learning Rate; Accuracy; Recall; Precision; F1score")

2018-11-20 18:24:02,319 - Xworkers_1mill_0 - INFO - Writing summaries to /home/ubuntu/summ_15ep_2wrk
2018-11-20 18:24:02,319 - Xworkers_1mill_0 - INFO - Writing summaries to /home/ubuntu/summ_15ep_2wrk
2018-11-20 18:24:02,322 - Xworkers_1mill_0 - INFO - Training
2018-11-20 18:24:02,322 - Xworkers_1mill_0 - INFO - Training
2018-11-20 18:24:02,323 - Xworkers_1mill_0 - INFO -   Step; Epoch; time-per-record(sec);  batchtime/worker(sec);  Loss;   Learning Rate; Accuracy; Recall; Precision; F1score
2018-11-20 18:24:02,323 - Xworkers_1mill_0 - INFO -   Step; Epoch; time-per-record(sec);  batchtime/worker(sec);  Loss;   Learning Rate; Accuracy; Recall; Precision; F1score


Checkpoint written to /home/ubuntu/summ_15ep_2wrk/checkpoint-0


In [28]:
def reset_and_update(sess, local_init, feed_dict):
    """Reset the local variables and update the necessary update ops."""
    # sess.run(local_init) # this is necesary in each batch??check out the local variables!
        
    update_names_list = [
        'metrics/auc/{:d}/hist_accumulate/update_op'.format(i)
        for i in range(7)
    ]

    update_names_list.extend([
        'metrics/m_measure/' + str(i) + str(j) + '/hist_accumulate/update_op'
        for i in range(7) for j in range(7) if i != j
    ])

    sess.run(update_names_list, feed_dict=feed_dict)
    return

In [29]:
def reshape_m_mtx(mtx):
    """Reshape the python list into a np array."""
    new_mtx = [0]
    for i in range(6):
        new_mtx.extend(mtx[i * 7:(i + 1) * 7])
        new_mtx.append(0)
    temp = np.array(new_mtx).reshape(7, 7)

    return temp

def print_stats(name, conf_mtx, accuracy, better_acc, precision, f1score_micro, f1score_macro, auc_list, auc_mean, m_list, m_list_mean, lloss, auc_pr, auc_pr_mean, loss):
    """Print to logger the given stats."""        
                
    m_mtx = np.nan_to_num(reshape_m_mtx(m_list)) 
    auc_list = np.nan_to_num(auc_list)
    conf_mtx = np.array(conf_mtx, dtype=int)
            
    stdout = 'Loss in ' + name +': {:.5f}\n'.format(loss)        
    stdout = stdout + ' Avg Log_Loss in ' + name +': {:.5f}\n'.format(lloss)
    stdout = stdout +  '{:s}:'.format(name) + ' (Silly) Global-ACC={:.5f}, Recall={:.5f},'.format(accuracy, better_acc) + \
        ' Avg M-Measure={:.4f},'.format(m_list_mean) + \
        ' Avg AUC_AOC={:.4f}'.format(auc_mean) + ' Avg AUC_PR={:.4f}\n'.format(auc_pr_mean)
    stdout = stdout + ' Precision= {:.5f} '.format(precision)
    stdout = stdout + ' f1score_micro= {:.5f} '.format(f1score_micro)
    stdout = stdout + ' f1score_macro= {:.5f}\n'.format(f1score_macro)
    stdout = stdout + (';').join(['Total Confusion Matrix', 'Total M-Measure Matrix', 'Total AUC_AOC', 'Total AUC_PR\n'])
    for conf_row, row, auc, auc_pr in zip(conf_mtx, m_mtx, auc_list, auc_pr):
        for conf_value in conf_row:
            stdout = stdout + '{}'.format(conf_value) + ';'
        stdout = stdout + ';'
        for value in row:
            stdout = stdout + '{:.4f}'.format(value) + ';'
        stdout = stdout + ';{:.4f}'.format(auc) + ' ;{:.4f}'.format(auc_pr) + '\n'
    stdout = stdout + '---------------------------------------------------------------------'
              
    logger.info('METRICS each %s (secs):  %s\r\n' % (FLAGS.summary_interval, stdout))


In [30]:
#for validation set:
def batching_dataset(sess, epoch, writer, tag, DATA, FLAGS):
    if tag =='valid':
        batch_num = DATA.validation.total_num_batch(FLAGS.valid_batch_size) 
        
    # metrics = acc_metrics_init(DATA)
    sess.run(local_init)
    start_time = datetime.now()
    acc_conf_mtx=np.zeros((DATA.train.num_classes, DATA.train.num_classes))        
    metrics = [0.0] * 7
    for batch_i in range(batch_num):
        step = epoch * batch_num + batch_i    # total steps for all epochs        
        feed = create_feed_dict(tag, DATA, FLAGS)     
        reset_and_update(sess, local_init, feed)
        summary, conf_mtx, accuracy, better_acc, precision, lloss, loss = sess.run([summary_ops, conf_mtx_op, accuracy_op, better_acc_op, precision_op, lloss_op, total_loss_op], feed_dict=feed)

        if (math.isnan(better_acc)):
            better_acc = calculate_better_acc(conf_mtx)

        if (math.isnan(precision)):
            precision = calculate_better_acc(conf_mtx, axis=0)
        
        f1score_micro = 2 * precision * better_acc / (precision + better_acc)
        f1score_macro = calculate_macro_f1score(np.array(conf_mtx, dtype='float32'))
        
        acc_conf_mtx = np.add(acc_conf_mtx, conf_mtx)
        metrics = np.add(metrics, np.array([accuracy, better_acc, precision, f1score_micro, f1score_macro, lloss, loss]))
        writer.add_summary(summary, step)
        writer.flush()
        del feed
    
    metrics[:] = [x / batch_num for x in metrics]
    valid_time = datetime.now() - start_time
    logger.info('%s - Number of batches: %d; batch_size: %d; Total Time: %s' %(tag, batch_num,  FLAGS.valid_batch_size, valid_time))
    return acc_conf_mtx, valid_time, metrics
    

In [31]:
if hvd.rank() == 0:
    if not FLAGS.eval:
        dtype = ['step','epoch','batch_time','Loss','LogLoss','Accuracy','Recall','Precision', 'f1score-Micro', 'f1score-Macro', 'M-Measure Mean','AUC_AOC Mean','AUC_PR Mean']        
        train_file = os.path.join(FLAGS.logdir, FLAGS.name + "_train.csv")
        valid_file = os.path.join(FLAGS.logdir, FLAGS.name + "_valid.csv")        
        
        if not Path(train_file).exists():
            df_train = pd.DataFrame(columns=dtype)                
            df_train.to_csv(train_file, sep=';', index=False)
        else:
            df_train = pd.read_csv(train_file, sep=';')

        if not Path(valid_file).exists():
            df_valid = pd.DataFrame(columns=dtype[:10])
            df_valid.to_csv(valid_file, sep=';', index=False)            
        else:
            df_valid = pd.read_csv(valid_file, sep=';')
        
        print('df_train: \n', df_train)
        print('df_valid: \n', df_valid)
        
    else:  # validation set:
        dtype = ['NN_name', 'NN_Number','Total Epochs', 'Execute Epochs', 'Total Training Time', 'Loss','LogLoss','Accuracy','Recall', 'Precision', 'f1score-Micro', 'f1score-Macro', 'M-Measure Mean','AUC_AOC Mean','AUC_PR Mean']


df_train: 
 Empty DataFrame
Columns: [step, epoch, batch_time, Loss, LogLoss, Accuracy, Recall, Precision, f1score-Micro, f1score-Macro, M-Measure Mean, AUC_AOC Mean, AUC_PR Mean]
Index: []
df_valid: 
 Empty DataFrame
Columns: [step, epoch, batch_time, Loss, LogLoss, Accuracy, Recall, Precision, f1score-Micro, f1score-Macro]
Index: []


In [32]:
def calculate_macro_f1score(conf_mtx):
    sum_byrows = conf_mtx.sum(axis=1, keepdims=True)
    recall = np.diag(np.divide(conf_mtx, sum_byrows, out=np.zeros_like(conf_mtx), where=sum_byrows!=0)) #, where=sum_byrows!=0
    #print('recall: ', recall)

    sum_bycols = np.sum(conf_mtx, axis=0, keepdims=True)
    #print('sum_bycols: ', sum_bycols)
    precision = np.diag(np.divide(conf_mtx, sum_bycols, out=np.zeros_like(conf_mtx), where=sum_byrows!=0)) #where=sum_bycols!=0
    #print('precision: ', precision)

    sum_rec_prec = recall + precision
    #print('sum_rec_prec: ', sum_rec_prec)
    f1sc_perclass = np.divide((2 * recall * precision), sum_rec_prec, out=np.zeros_like(recall), where=sum_rec_prec!=0)


    #print('f1sc_perclass: ', f1sc_perclass)
    np.nan_to_num(f1sc_perclass, copy=False)
    #if (recall==np.nan() or precision==np.nan()):
    micro_f1sc = np.average(f1sc_perclass)
    #print('micro_f1sc: ', micro_f1sc)
    
    return micro_f1sc

In [33]:
ops_to_run = [learning_rate_op, train_ops]
ops_stats = [conf_mtx_op, accuracy_op, better_acc_op, precision_op, auc_list_op, auc_mean_op, m_list_op, m_list_mean_op, 
             lloss_op, auc_pr_op, auc_pr_mean_op, total_loss_op]
                    
oom = False
step0 = int(sess.run(trainer.global_step))
for step in range(step0, nstep):    
    try:
        start_time = time.time()
        epoch = step*FLAGS.batch_size // nrecord #*hvd.size()
        batch_dict= create_feed_dict('batch', DATA, FLAGS)        
        
        if (hvd.rank() == 0 and summary_ops is not None and
            (step == 0 or step+1 == nstep or
             time.time() - last_summary_time > FLAGS.summary_interval)):
            
            if step != 0:
                last_summary_time += FLAGS.summary_interval                        
                
            reset_and_update(sess, local_init, batch_dict)
            summary, conf_mtx, accuracy, better_acc, precision, auc_list, auc_mean, m_list, m_list_mean, lloss, auc_pr, auc_pr_mean, loss, lr, _ = sess.run([summary_ops] + ops_stats + ops_to_run, feed_dict=batch_dict)                        
            train_writer.add_summary(summary, step)            
            train_writer.flush()
            
            if (math.isnan(better_acc)):
                better_acc = calculate_better_acc(conf_mtx)        
            if (math.isnan(precision)):
                precision = calculate_better_acc(conf_mtx, axis=0)                         
            f1score_micro = 2 * precision * better_acc / (precision + better_acc)
            f1score_macro = calculate_macro_f1score(np.array(conf_mtx, dtype='float32'))
            
            elapsed = time.time() - start_time            
            #this not necessarily matches with the display at console not even with validation set, due the summary_interval!
            print_stats('---Training in Summary---', conf_mtx, accuracy, better_acc, precision, f1score_micro, f1score_macro, auc_list, auc_mean, m_list, m_list_mean, lloss, auc_pr, auc_pr_mean, loss)                         
            df_train.loc[len(df_train)] = [step+1, epoch+1, elapsed, loss, lloss, accuracy, better_acc, precision, f1score_micro, f1score_macro, m_list_mean, auc_mean, auc_pr_mean]                                                    
        else:
            accuracy, conf_mtx, better_acc, precision, loss, lr, _ = sess.run([accuracy_op, conf_mtx_op, better_acc_op, precision_op, total_loss_op] + ops_to_run, feed_dict=batch_dict)
            
            if (math.isnan(better_acc)):
                better_acc = calculate_better_acc(conf_mtx)        
            if (math.isnan(precision)):
                precision = calculate_better_acc(conf_mtx, axis=0)            
            f1score_micro = 2 * precision * better_acc / (precision + better_acc)
            f1score_macro = calculate_macro_f1score(np.array(conf_mtx, dtype='float32'))
            
            elapsed = time.time() - start_time
            
        if step == 0 or (step+1) % FLAGS.display_every == 0:                    
            feature_per_sec = FLAGS.batch_size / elapsed                        
            logger.info("%6i; %5i; %7.1f; %7.3f; %7.5f; %7.5f; %7.5f; %7.5f; %7.5f; %7.5f; %7.5f" % (
                step+1, epoch+1, feature_per_sec*hvd.size(), elapsed, loss, lr, accuracy, better_acc, precision, f1score_micro, f1score_macro))        

        if (hvd.rank() == 0 and  ((step+1) % nstep_per_epoch == 0 or step+1 == nstep)):
            #Running validation set:
            valid_conf_mtx, valid_time, metrics = batching_dataset(sess, epoch, valid_writer, 'valid', DATA, FLAGS)
            #valid_conf_mtx = np.array2string(valid_conf_mtx, formatter={'int_type':lambda x: "int(%)" % x})
            valid_conf_mtx = np.array(valid_conf_mtx, dtype=int)
            logger.info("---Validation--- Training Step: %d; Training Epoch: %d; \n Confusion Matrix:\n %s" % (step+1, epoch+1, str(valid_conf_mtx)))            
            df_valid.loc[len(df_valid)] = [step+1, epoch+1, valid_time, metrics[6], metrics[5], metrics[0], metrics[1], metrics[2], metrics[3], metrics[4]]            
            logger.info("(Training Step, Training Epoch, loss, logloss, accuracy, recall, precision, f1score_micro, f1score_macro) in Validation: %6i; %5i; %7.5f; %7.5f; %7.5f; %7.5f; %7.5f; %7.5f; %7.5f" % (
                step+1, epoch+1, metrics[6], metrics[5], metrics[0], metrics[1], metrics[2], metrics[3], metrics[4]))    
            sess.run(local_init)
        
        del batch_dict
                    
    except KeyboardInterrupt:
        if hvd.rank() == 0:
            df_train.to_csv(train_file, index=False, mode='a', sep =';', header=False)
            df_valid.to_csv(valid_file, index=False, mode='a', sep =';', header=False)
        logger.info("Keyboard interrupt")
        break
    except tf.errors.ResourceExhaustedError:
        elapsed = -1.
        loss    = 0.
        lr      = -1
        if hvd.rank() == 0:
            df_train.to_csv(train_file, index=False, mode='a', sep =';', header=False)
            df_valid.to_csv(valid_file, index=False, mode='a', sep =';', header=False)
        oom = True
    
    if (hvd.rank() == 0 and saver is not None and
        (time.time() - last_save_time > FLAGS.save_interval or step+1 == nstep)):
        last_save_time += FLAGS.save_interval
        save_path = saver.save(sess, checkpoint_file, global_step=trainer.global_step)
        print("Checkpoint written to", save_path)
    
    if oom:
        break
        

if hvd.rank() == 0:                               
    df_train.to_csv(train_file, index=False, mode='a', sep =';', header=False)
    df_valid.to_csv(valid_file, index=False, mode='a', sep =';', header=False)
                               
if train_writer is not None:
    train_writer.close()

if valid_writer is not None:
    valid_writer.close()    
    
global_end_time = time.time()
#logger.info("start time is {}, end time is {}".format(global_start_time, global_end_time))
logger.info('Time used in total: %.1f seconds' % (global_end_time - global_start_time))

if oom:
    print("Out of memory error detected, exiting")
    sys.exit(-2)
        

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:8: RuntimeWarning: invalid value encountered in true_divide
2018-11-20 18:26:57,034 - Xworkers_1mill_0 - INFO - METRICS each 1800 (secs):  Loss in ---Training in Summary---: 0.42526
 Avg Log_Loss in ---Training in Summary---: 0.51851
---Training in Summary---: (Silly) Global-ACC=0.02960, Recall=0.13069, Avg M-Measure=0.4975, Avg AUC_AOC=0.4998 Avg AUC_PR=0.0000
 Precision= 0.13441  f1score_micro= 0.13252  f1score_macro= 0.01423
Total Confusion Matrix;Total M-Measure Matrix;Total AUC_AOC;Total AUC_PR
0;39;6;19;0;0;1;;0.0000;0.3733;0.3980;0.3041;0.4197;0.2894;0.5423;;0.4135 ;0.0000
0;88;18;11;1;0;1;;0.5930;0.0000;0.5124;0.4926;0.5510;0.5498;0.7017;;0.5499 ;0.0000
0;31;5;3;0;0;0;;0.5288;0.4866;0.0000;0.4338;0.5426;0.4127;0.3045;;0.5357 ;0.0000
0;54;20;3;1;0;0;;0.3595;0.4340;0.4198;0.0000;0.3658;0.5610;0.5865;;0.3720 ;0.0000
0;2695;422;823;35;0;66;;0.4727;0.5126;0.4487;0.5697;0.0000;0.6301;0.5863;;

2018-11-20 18:29:18,733 - Xworkers_1mill_0 - INFO -   2000;     1; 131453.1;   0.067; 0.11782; 0.09994; 0.91774; 0.51900; 0.50257; 0.51066; 0.48908
2018-11-20 18:29:18,733 - Xworkers_1mill_0 - INFO -   2000;     1; 131453.1;   0.067; 0.11782; 0.09994; 0.91774; 0.51900; 0.50257; 0.51066; 0.48908
2018-11-20 18:29:25,674 - Xworkers_1mill_0 - INFO -   2100;     1; 125138.5;   0.071; 0.12756; 0.09994; 0.91729; 0.50190; 0.50953; 0.50569; 0.49095
2018-11-20 18:29:25,674 - Xworkers_1mill_0 - INFO -   2100;     1; 125138.5;   0.071; 0.12756; 0.09994; 0.91729; 0.50190; 0.50953; 0.50569; 0.49095


Checkpoint written to /home/ubuntu/summ_15ep_2wrk/checkpoint-2144


2018-11-20 18:29:33,597 - Xworkers_1mill_0 - INFO -   2200;     1; 127737.4;   0.069; 0.12360; 0.09994; 0.91706; 0.53563; 0.52465; 0.53008; 0.51058
2018-11-20 18:29:33,597 - Xworkers_1mill_0 - INFO -   2200;     1; 127737.4;   0.069; 0.12360; 0.09994; 0.91706; 0.53563; 0.52465; 0.53008; 0.51058
2018-11-20 18:29:40,508 - Xworkers_1mill_0 - INFO -   2300;     1; 130161.1;   0.068; 0.11034; 0.09993; 0.92384; 0.52527; 0.53273; 0.52897; 0.50819
2018-11-20 18:29:40,508 - Xworkers_1mill_0 - INFO -   2300;     1; 130161.1;   0.068; 0.11034; 0.09993; 0.92384; 0.52527; 0.53273; 0.52897; 0.50819
2018-11-20 18:29:47,391 - Xworkers_1mill_0 - INFO -   2400;     1; 128740.4;   0.069; 0.12868; 0.09993; 0.92497; 0.53754; 0.58201; 0.55889; 0.52787
2018-11-20 18:29:47,391 - Xworkers_1mill_0 - INFO -   2400;     1; 128740.4;   0.069; 0.12868; 0.09993; 0.92497; 0.53754; 0.58201; 0.55889; 0.52787
2018-11-20 18:29:54,290 - Xworkers_1mill_0 - INFO -   2500;     1; 122530.4;   0.072; 0.11883; 0.09993; 0.91548;

2018-11-20 18:32:47,158 - Xworkers_1mill_0 - INFO -   5000;     1; 132060.1;   0.067; 0.11900; 0.09986; 0.91910; 0.53045; 0.53556; 0.53299; 0.50487
2018-11-20 18:32:47,158 - Xworkers_1mill_0 - INFO -   5000;     1; 132060.1;   0.067; 0.11900; 0.09986; 0.91910; 0.53045; 0.53556; 0.53299; 0.50487
2018-11-20 18:32:54,067 - Xworkers_1mill_0 - INFO -   5100;     1; 125392.6;   0.071; 0.10956; 0.09986; 0.93175; 0.53473; 0.57399; 0.55366; 0.52772
2018-11-20 18:32:54,067 - Xworkers_1mill_0 - INFO -   5100;     1; 125392.6;   0.071; 0.10956; 0.09986; 0.93175; 0.53473; 0.57399; 0.55366; 0.52772
2018-11-20 18:33:00,966 - Xworkers_1mill_0 - INFO -   5200;     1; 124094.3;   0.071; 0.12270; 0.09985; 0.92090; 0.54983; 0.57887; 0.56398; 0.53365
2018-11-20 18:33:00,966 - Xworkers_1mill_0 - INFO -   5200;     1; 124094.3;   0.071; 0.12270; 0.09985; 0.92090; 0.54983; 0.57887; 0.56398; 0.53365
2018-11-20 18:33:07,853 - Xworkers_1mill_0 - INFO -   5300;     1; 128465.0;   0.069; 0.11199; 0.09985; 0.92181;

...

2018-11-20 20:10:39,658 - Xworkers_1mill_0 - INFO -  85100;    15; 136633.3;   0.065; 0.09798; 0.09764; 0.92746; 0.58491; 0.66641; 0.62301; 0.58226
2018-11-20 20:10:39,658 - Xworkers_1mill_0 - INFO -  85100;    15; 136633.3;   0.065; 0.09798; 0.09764; 0.92746; 0.58491; 0.66641; 0.62301; 0.58226
2018-11-20 20:10:46,009 - Xworkers_1mill_0 - INFO -  85200;    15; 138983.0;   0.064; 0.10954; 0.09764; 0.91864; 0.60050; 0.67906; 0.63737; 0.59905
2018-11-20 20:10:46,009 - Xworkers_1mill_0 - INFO -  85200;    15; 138983.0;   0.064; 0.10954; 0.09764; 0.91864; 0.60050; 0.67906; 0.63737; 0.59905
2018-11-20 20:10:52,418 - Xworkers_1mill_0 - INFO -  85300;    15; 137762.6;   0.064; 0.09847; 0.09763; 0.92136; 0.61976; 0.66556; 0.64184; 0.61064
2018-11-20 20:10:52,418 - Xworkers_1mill_0 - INFO -  85300;    15; 137762.6;   0.064; 0.09847; 0.09763; 0.92136; 0.61976; 0.66556; 0.64184; 0.61064
2018-11-20 20:10:58,802 - Xworkers_1mill_0 - INFO -  85400;    15; 139849.1;   0.063; 0.10306; 0.09763; 0.92045;

2018-11-20 20:13:37,841 - Xworkers_1mill_0 - INFO -  87900;    15; 132383.7;   0.067; 0.10889; 0.09756; 0.92316; 0.62753; 0.66507; 0.64576; 0.62627
2018-11-20 20:13:37,841 - Xworkers_1mill_0 - INFO -  87900;    15; 132383.7;   0.067; 0.10889; 0.09756; 0.92316; 0.62753; 0.66507; 0.64576; 0.62627
2018-11-20 20:13:44,242 - Xworkers_1mill_0 - INFO -  88000;    15; 138163.6;   0.064; 0.10341; 0.09756; 0.91593; 0.55952; 0.51074; 0.53402; 0.52180
2018-11-20 20:13:44,242 - Xworkers_1mill_0 - INFO -  88000;    15; 138163.6;   0.064; 0.10341; 0.09756; 0.91593; 0.55952; 0.51074; 0.53402; 0.52180
2018-11-20 20:13:50,677 - Xworkers_1mill_0 - INFO -  88100;    15; 137319.8;   0.064; 0.10943; 0.09756; 0.91480; 0.59500; 0.68125; 0.63521; 0.59610
2018-11-20 20:13:50,677 - Xworkers_1mill_0 - INFO -  88100;    15; 137319.8;   0.064; 0.10943; 0.09756; 0.91480; 0.59500; 0.68125; 0.63521; 0.59610
2018-11-20 20:14:35,758 - Xworkers_1mill_0 - INFO - valid - Number of batches: 1; batch_size: 1200000; Total Tim

Checkpoint written to /home/ubuntu/summ_15ep_2wrk/checkpoint-88125


2018-11-20 20:14:37,921 - Xworkers_1mill_0 - INFO - METRICS each 1800 (secs):  Loss in ---Training in Summary---: 0.09514
 Avg Log_Loss in ---Training in Summary---: 0.07914
---Training in Summary---: (Silly) Global-ACC=0.92542, Recall=0.64452, Avg M-Measure=0.9453, Avg AUC_AOC=0.9475 Avg AUC_PR=0.5775
 Precision= 0.68481  f1score_micro= 0.66406  f1score_macro= 0.64002
Total Confusion Matrix;Total M-Measure Matrix;Total AUC_AOC;Total AUC_PR
21;1;0;2;44;1;0;;0.0000;0.9354;0.9745;0.9832;0.8245;0.9782;0.9738;;0.8357 ;0.3306
0;80;5;5;30;0;0;;0.9420;0.0000;0.6043;0.9680;0.9124;0.9943;0.9993;;0.9133 ;0.3143
0;21;12;6;0;0;0;;0.9938;0.8406;0.0000;0.9274;0.9963;0.9923;0.9999;;0.9891 ;0.3755
0;0;7;91;0;9;0;;0.9860;0.9875;0.9433;0.0000;0.9992;0.9163;0.9565;;0.9965 ;0.8164
29;154;3;5;3823;2;0;;0.8072;0.9133;0.9958;0.9991;0.0000;0.9996;0.9998;;0.9426 ;0.9896
0;0;0;5;0;67;0;;0.9936;0.9977;0.9941;0.9284;0.9997;0.0000;0.7053;;0.9969 ;0.8399
0;0;0;0;0;1;1;;0.9943;0.9998;0.9998;0.9617;0.9999;0.7852;0.00

Checkpoint written to /home/ubuntu/summ_15ep_2wrk/checkpoint-88136


In [ ]:
!jupyter nbconvert --to=python  nn_real_hvd-ntb-v7-rel.ipynb

In [ ]:
!scp nn_real_hvd-ntb-v7-rel.py ubuntu@ec2-54-172-15-153.compute-1.amazonaws.com:/home/ubuntu/MLMortgage/notebooks/

In [ ]:
!scp /home/ubuntu/MLMortgage/src/data/data_classes.py ubuntu@ec2-18-208-144-183.compute-1.amazonaws.com:/home/ubuntu/MLMortgage/src/data/

In [ ]:
!mpirun -np 9 -H ec2-18-208-144-183.compute-1.amazonaws.com,ec2-54-163-27-28.compute-1.amazonaws.com,ec2-54-89-216-7.compute-1.amazonaws.com,ec2-18-234-208-171.compute-1.amazonaws.com,ec2-18-233-102-157.compute-1.amazonaws.com,ec2-34-238-152-99.compute-1.amazonaws.com,ec2-52-87-79-207.compute-1.amazonaws.com,ec2-54-165-210-130.compute-1.amazonaws.com,ec2-54-243-9-240.compute-1.amazonaws.com --mca plm_rsh_no_tree_spawn 1 --prefix /usr/local/mpi --bind-to none --map-by slot -x NCCL_DEBUG=INFO -x NCCL_MIN_NRINGS=2 -x LD_LIBRARY_PATH -x PATH  -mca pml ob1 -mca btl ^openib python nn_real_hvd-ntb-v7-rel.py